# Generator Danych Testowych

## Import modułów i bibliotek

Skrypt wykorzystuje następujące moduły i biblioteki
- [Faker](https://faker.readthedocs.io/en/master/),
- [random](https://docs.python.org/3/library/random.html),
- [pandas](https://pandas.pydata.org/docs/).

In [1]:
from faker import Faker
import random as rd
import pandas as pd
fake = Faker("pl_PL")

## Podstawowe ustawienia

W słowniku `city` można ustawić nazwę (`name`) gminy, dla której chcemy wygenerować dane oraz zasięg (`y_min_max` oraz `x_min_max`), w którym mają zosta wygnerowane współrzędne punktowe. Zasięg można obliczyć w QGIS na podstawie granic gminy. Dane dla każdego z wymiarów podawane się w krotce (tuple), w której piewsza liczba to minimum a druga maksimum.

Krotka `sample_categories` zawiera nazwy kategorii, które będą wykorzystane przy generowaniu danych.

Zmienna `desired_samples` określa liczbę próbek, które mają zostać wygenerowane. **NB.** Należy wpisać 50%-60% więcej niż ma się faktycznie znaleźć w granicach. Po wygenrowaniu pliku, należy go przyciąć do granic w QGIS.

Zmienna `dev_mode` to flaga. Jeżeli ustawiona jako prawda (`True`), plik csv nie zostanie zapisany, w przeciwnym wypadku (`False`) plik zostanie zapisany.

In [2]:
city:dict[str, str | tuple[float | int]] = {
    "name": "Siemianowice Śląskie",
    "y_min_max": (50.274825, 50.349919),
    "x_min_max": (18.98523, 19.061271)
}

sample_categories = ("administracja", "dostępność", "infrastruktura", "kultura, sport i rekreacja", "ochrona środowiska", "oświata", "transport")

desired_samples:int = 200

dev_mode = False

## Generowanie danych

### Generowanie współrzędnych x i y

In [3]:
def generate_coords(y_or_x:str, num_to_generate:int = desired_samples, extent_y:tuple[float | int ] = city["y_min_max"], extent_x:tuple[float | int] = city["x_min_max"]) -> list[float | None]:
    match y_or_x:
        case "y":
            y:list[float] = []
            min_y, max_y = extent_y
            for _ in range(num_to_generate):
                y.append(rd.uniform(min_y, max_y))
            return y
        case "x":
            x:list[float] = []
            min_x, max_x = extent_x
            for _ in range(num_to_generate):
                x.append(rd.uniform(min_x, max_x))
            return x
        # in any other case return an empty list:    
        case _:
            return [None] * num_to_generate

### Generowanie pozostałych danych

In [4]:
def generate_test_data(data_type:str, num_to_generate:int = desired_samples, max_num_chars:int = 256, no_dot_at_end:bool = True, single_city:bool = True) -> list:
    match data_type:
        case "texts":
            texts:list[str] = fake.texts(nb_texts=num_to_generate, max_nb_chars=max_num_chars)
            if no_dot_at_end:
                for i in range(len(texts)):
                    texts[i] = texts[i][:-1]
            return texts
        case "categories":
            return fake.random_choices(elements=sample_categories, length=num_to_generate)
        case "prefixes":
            return fake.random_choices(elements=("ul.", "al.", "pl."), length=num_to_generate)
        case "streets":
            s = []
            for _ in range(num_to_generate):
                s.append(fake.street_name())
            return s
        case "buildings":
            b = []
            for _ in range(num_to_generate):
                if rd.random() > 0.33:
                    b.append(fake.building_number())
                else:
                    b.append(None)
            return b
        case "postcodes":
            p:list[str] = []
            for _ in range(num_to_generate):
                p.append(fake.postcode())
            return p
        case "cities":
            c = []
            if single_city:
                c.append(city["name"])
                return c * num_to_generate
            else:
                for _ in range(num_to_generate):
                    c.append(fake.city())
                return c
        case "dates":
            d = []
            for _ in range(num_to_generate):
                d.append(fake.date_between(start_date="-10y", end_date="-6m"))
            return d
        case "years":
            y = []
            for _ in range(num_to_generate):
                y.append(fake.year())
            return y
        case "amounts":
            a = []
            for _ in range(num_to_generate):
                a.append(fake.pyfloat(left_digits=7, right_digits=2, positive=True))
            return a
        case "bools":
            b = []
            for _ in range(num_to_generate):
                b.append(fake.pybool())
            return b
        case "urls":
            u:list[str] = []
            domain_name:str = fake.safe_domain_name()
            for _ in range(num_to_generate):
                if rd.random() > 0.25:
                    u.append(f"https://{domain_name}/{fake.slug()}")
                else:
                    u.append(None)
            return u
        case _:
            return [None] * num_to_generate

### Struktura tabeli

Słownik `d` zawiera strukturę generowanej tabeli. Każdy element to kolumna tabeli.

In [5]:
d = {
    "foto_sciezka": [None] * desired_samples, # to be filled in in QGIS with image path
    "kategoria": generate_test_data("categories"),
    "nazwa": generate_test_data("texts", max_num_chars = 64),
    "zakres": generate_test_data("texts", max_num_chars = 256, no_dot_at_end = False),
    "cecha": generate_test_data("prefixes"),
    "ulica_nazwa": generate_test_data("streets"),
    "nr_budynku": generate_test_data("buildings"), # will be empty for approx 1/3 samples, this is expected behaviour
    "kod_pocztowy": generate_test_data("postcodes"),
    "miejscowosc": generate_test_data("cities", single_city = True), # if multiple city names needed, change `single_city` to False
    "adres": [None] * desired_samples, # to be filled in at the data cleaning stage
    "data_rozpoczecia": generate_test_data("dates"),
    "data_zakonczenia": generate_test_data("dates"), # to be cleaned & adjusted at the data cleaning stage
    "czy_zakonczony": generate_test_data("bools"),
    "wartosc": generate_test_data("amounts"),
    "czy_dofinansowany": generate_test_data("bools"),
    "zrodlo_dofinansowania": generate_test_data("texts", max_num_chars = 64), # to be cleaned at the data cleaning stage
    "wartosc_dofinansowania": generate_test_data("amounts"), # to be cleaned & adjusted at the data cleaning stage
    "czy_bo": generate_test_data("bools"),
    "edycja_bo": generate_test_data("years"), # to be cleaned at the data cleaning stage
    "url_fiszka": generate_test_data("urls"), # will be empty for approx 1/4 samples, this is expected behaviour
    "url_geoportal": generate_test_data("urls"), # will be empty for approx 1/4 samples, this is expected behaviour
    "url_mapa": generate_test_data("urls"), # will be empty for approx 1/4 samples, this is expected behaviour
    "popup_url_html": [None] * desired_samples, # to be filled in at the data cleaning stage
    "y": generate_coords("y"),
    "x": generate_coords("x")
}

## Oczyszczanie danych testowych

Wygenerowane dane wymagają oczyszczenie i uzupełnień przed eksportem.

Funkcja `fill_address_column()` populuje kolumnę `adres` na podstawie wygenerowanych danych cząstkowych.

Funkcja `adjust_aid_received()` dostosowuje wartość dofinansowania.

Funkcja `adjust_end_dates()` dostosowuje daty zakończenia.

Funkcja `fill_popup_url_html()` generuje kod html dla linków, który będzie wyświetlony w popupie.

Funkcja `clean_unwanted_data()` usuwa wartości, które powinny być puste (np. datę zakończenia niezakończonych inwestycji).

In [6]:
def fill_address_column() -> None:
    for index in range(len(df.index)):
        if df.at[index, "nr_budynku"] != None:
            full_address:str = f'{df.at[index, "cecha"]} {df.at[index, "ulica_nazwa"]} {df.at[index, "nr_budynku"]}; {df.at[index, "kod_pocztowy"]} {df.at[index, "miejscowosc"]}'
        else:
            full_address:str = f'{df.at[index, "cecha"]} {df.at[index, "ulica_nazwa"]}; {df.at[index, "kod_pocztowy"]} {df.at[index, "miejscowosc"]}'
        df.at[index, "adres"] = full_address

def adjust_aid_received(min_spread:float = 0.65, max_spread:float = 0.85) -> None:
    for index in range(len(df.index)):
        df.at[index, "wartosc_dofinansowania"] = round(df.at[index, "wartosc"] * rd.uniform(min_spread, max_spread), 2)

def adjust_end_dates() -> None:
    for index in range(len(df.index)):
        if df.at[index, "data_zakonczenia"] < df.at[index, "data_rozpoczecia"]:
            df.at[index, "data_zakonczenia"] = fake.date_between_dates(date_start = df.at[index, "data_rozpoczecia"], date_end = "-1m")

def fill_popup_url_html() -> None:
    for index in range(len(df.index)):
        output_html:str | None = ""
        links:list[tuple[str]] = [(df.at[index, "url_fiszka"], "Więcej"), (df.at[index, "url_geoportal"], "Geoportal"), (df.at[index, "url_mapa"], "Mapa")]
        if (links[0][0] == None and links[1][0] == None and links[2][0] == None):
            output_html = None
        else:
            for link in links:
                url, title = link
                if url != None:
                    output_html += f'<a href="{url}">{title}</a>'
                    output_html = output_html.replace("</a><a", "</a> | <a")
                    if len(output_html) > 256:
                        output_html = None
        df.at[index, "popup_url_html"] = output_html

def clean_unwanted_data() -> None:
    df.loc[df["czy_zakonczony"] == False, "data_zakonczenia"] = None
    df.loc[df["czy_dofinansowany"] == False, "zrodlo_dofinansowania"] = None
    df.loc[df["czy_dofinansowany"] == False, "wartosc_dofinansowania"] = None
    df.loc[df["czy_bo"] == False, "edycja_bo"] = None

In [7]:
df = pd.DataFrame(data=d)
fill_address_column()
adjust_aid_received()
adjust_end_dates()
fill_popup_url_html()
clean_unwanted_data()
df

,foto_sciezka,kategoria,nazwa,zakres,cecha,ulica_nazwa,nr_budynku,kod_pocztowy,miejscowosc,adres,...,zrodlo_dofinansowania,wartosc_dofinansowania,czy_bo,edycja_bo,url_fiszka,url_geoportal,url_mapa,popup_url_html,y,x
0,None,transport,Trwały mieszkanka dany znaczyć. Okropny – dzie...,Uczyć Się literatura właściciel dużo obiekt. J...,pl.,Wysoka,07/13,64-194,Siemianowice Śląskie,pl. Wysoka 07/13; 64-194 Siemianowice Śląskie,...,Armia zachowywać się problem obraz opłata gaze...,3813517.91,True,1995,https://example.com/finansowy-zgoda,https://example.net/bitwa-majuskua,https://example.com/tysiac-warstwa-god,"<a href=""https://example.com/finansowy-zgoda"">...",50.277199,19.013300
1,None,"kultura, sport i rekreacja",Informacja właśnie gałąź strata rodzaj leczeni...,Deszcz dodatkowy rozpocząć wysyłać suma. Położ...,pl.,Grunwaldzka,None,02-636,Siemianowice Śląskie,pl. Grunwaldzka; 02-636 Siemianowice Śląskie,...,None,NaN,False,None,https://example.com/ocena,None,https://example.com/czasownik-oddac,"<a href=""https://example.com/ocena"">Więcej</a>...",50.281300,19.049709
2,None,infrastruktura,Głupota masa rzadki zakres amharski. Ładny go ...,Pająk tłum lista aktywny nasz gałąź bóg para. ...,al.,Orzechowa,None,58-582,Siemianowice Śląskie,al. Orzechowa; 58-582 Siemianowice Śląskie,...,Publiczny potrzebować pochodzenie wy działalność,1087000.47,True,1991,None,None,https://example.com/gupota-uzycie,"<a href=""https://example.com/gupota-uzycie"">Ma...",50.307926,19.000325
3,None,administracja,Dokument lekarz łatwo rzucić prawie nasz,Pogoda tarcza mieszkaniec kraina nauczycielka....,pl.,Irysowa,67,44-626,Siemianowice Śląskie,pl. Irysowa 67; 44-626 Siemianowice Śląskie,...,None,NaN,False,None,https://example.com/czoo-upadek,https://example.net/pacjent-rzymski,https://example.com/walczyc-stary,"<a href=""https://example.com/czoo-upadek"">Więc...",50.343679,18.991513
4,None,oświata,Środa obok wnętrze napisać społeczny godny,Pokazywać 8 jajo medycyna rosnąć nowy. Gdyby 6...,al.,Browarna,None,14-324,Siemianowice Śląskie,al. Browarna; 14-324 Siemianowice Śląskie,...,None,NaN,True,2023,https://example.com/zimny-wasnie-tez,https://example.net/wiek-wyrazenie,https://example.com/skadac-kosz-grob,"<a href=""https://example.com/zimny-wasnie-tez""...",50.346321,19.047698
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,None,administracja,Sportowy odbywać się liczny plan,Aktor włoski płyta system samolot. Składać usł...,pl.,Chabrowa,07/55,19-347,Siemianowice Śląskie,pl. Chabrowa 07/55; 19-347 Siemianowice Śląskie,...,None,NaN,False,None,https://example.com/wyjsc-az-prawda,https://example.net/publiczny-kieszen,https://example.com/niedzwiedz-tam,"<a href=""https://example.com/wyjsc-az-prawda"">...",50.304618,19.058555
196,None,administracja,Sportowy zespół powietrze uciec walka,Prędkość sowa 3 poza 7 artysta. Piłka kolej dr...,ul.,Plażowa,None,91-924,Siemianowice Śląskie,ul. Plażowa; 91-924 Siemianowice Śląskie,...,Klub postępowanie broń delikatny kraina ale,1816917.69,True,2011,https://example.com/uprawiac-jeszcze,https://example.net/kamien-ogromny-obcy,None,"<a href=""https://example.com/uprawiac-jeszcze""...",50.333748,19.037305
197,None,"kultura, sport i rekreacja",Nagle ziemski plan gdy kość,Prasa głupota święto królewski nieprzyjemny po...,al.,Strzelecka,96,44-423,Siemianowice Śląskie,al. Strzelecka 96; 44-423 Siemianowice Śląskie,...,None,NaN,False,None,https://example.com/nad-nasz-spac,https://example.net/dziaac-wiele,https://example.com/chmura-uczen,"<a href=""https://example.com/nad-nasz-spac"">Wi...",50.288784,18.990406
198,None,oświata,Gospodarczy stopa specjalny metalowy wybór,Mężczyzna stać się szczyt sobie ochrona szpita...,pl.,Nowa,28,06-896,Siemianowice Śląskie,pl. Nowa 28; 06-896 Siemianowice Śląskie,...,Słodki strach deska rano funkcja mężczyzna zaj...,2983451.00,False,None,https://example.com/pora-klasa,None,https://example.com/chinski-psychiczny,"<a href=""https://example.com/pora-klasa"">Więce.

## Eksport danych testowych do pliku

Jeżeli dane się nie zapisują, sprawdź wartość flagi `dev_mode` w sekcji *Podstawowe ustawienia* - powinna zostać ustawiona, jako `False`.

In [8]:
if dev_mode == False:
    df.to_csv('test_data_set.csv', index=False)